# Heteroskedastic
meaning not i.i.d noise
- GPs do not typically cope with heteroskedasticity and instead predict the same noise level across the whole domain
- it is unclear which of the two solutions in the function below is preferable, for situations where robust performance is required, then the left peak is preferable to the right peak. Without extensions, Bayesian optimisation will usually favour the right peak
- for applications, see 'Variational Bayesian Optimization for Runtime Risk-Sensitive Control'

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import sklearn.gaussian_process as sk_gp
import GPy
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() # prettify matplotlib

In [ ]:
# local modules
import turbo as tb
import turbo.modules as tm
import turbo.plotting as tp
import turbo.gui.jupyter as tg

In [ ]:
# make deterministic
np.random.seed(42)

In [ ]:
def RBF(x, c, beta):
    return np.exp(-beta * (x-c)**2)

#noise_f = lambda x: 0.1*RBF(x, -5, 1e-3) + 0.1*RBF(x, 5, 5e-3) + 0.1*RBF(x, 18, 5e-3) + 0.1
noise_f = lambda x: 0.2*RBF(x, 15, 4e-3)
def f(x):
    noise = np.random.normal(loc=0, scale=noise_f(x))
    v = 2 * RBF(x, 1, 1e-3) + 1 * RBF(x, -10, 0.2) + 1 * RBF(x, 10, 0.2)
    return v + noise

xmin, xmax = -20, 20
xs = np.linspace(xmin, xmax, num=1000)

ys = f(xs)
best_y = np.max(ys)
best_x = xs[np.argmin(ys)]

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(xs, ys, label='f')
#plt.plot(xs, noise_f(xs), label='noise')
plt.show()

In [ ]:
op = tb.Optimiser(f, 'max', [('x', xmin, xmax)], pre_phase_trials=5, settings_preset='default')

#op.acquisition = tm.UCB(beta=lambda trial_num: 5*np.log(trial_num-2)-4)
#tp.plot_acquisition_parameter_function(op.acquisition.beta, 5, 30)

op.surrogate = tm.SciKitGPSurrogate(model_params=dict(
    kernel = 1.0 * sk_gp.kernels.Matern(nu=2.5) + sk_gp.kernels.WhiteKernel(),
), training_iterations=4)
rec = tb.Recorder(op)

In [ ]:
tg.OptimiserProgressBar(op)
op.run(max_trials=30)

In [ ]:
tp.plot_error(rec, true_best=best_y);
tp.plot_overview(rec)

In [ ]:
tp.interactive_plot_trial_1D(rec, true_objective=f)